#**Week16_예습과제_우정연**

##**9.4 잠재 요인 협업 필터링**

- SGD(Stochastic Gradient Descent: 확률적 경사 하강법)을 이용해 행렬 분해를 수행하는 예제
  - 분해하려는 원본 행렬 R을 P와 Q로 분해한 뒤에 다시 P와 Q.T의 내적으로 예측 행렬을 만드는 예제

In [1]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])

num_users, num_items = R.shape
K = 3

# P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력
np.random.seed(1)
P = np.random.normal(scale = 1./K, size = (num_users, K))
Q = np.random.normal(scale = 1./K, size = (num_items, K))


- 실제 R 행렬과 예측 행렬의 오차를 구하는 `get_rmse()` 함수
  - `get_rmse()` 함수: 실제 R 행렬의 널이 아닌 행렬 값의 위치 인덱스를 추출해 이 인덱스에 있는 실제 R 행렬 값과 분해된 P, Q를 이용해 다시 조합된 예측 행렬 값의 RMSE 값을 반환

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
  error = 0
  # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
  full_pred_matrix = np.dot(P, Q.T)

  # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
  x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
  y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
  R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
  full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
  mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
  rmse = np.sqrt(mse)

  return rmse

- SGD 기반으로 행렬 분석 수행
  - R에서 널 값을 제외한 데이터의 행렬 인덱스 추출
  - steps는 SGD를 반복해서 업데이트할 횟수, learning_rate은 SGD의 학습률, r_lambda는 L2 Regularization 계수
  - steps = 1000 동안 반복하면서 새로운 p,q값으로 업데이트
  - get_rmse() 함수를 통해 50회 반복시마다 오류 값을 출력

In [3]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [ (i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0 ]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
  for i, j, r in non_zeros:
    # 실제 값과 예측 값의 차이인 오류 값 구함
    eji = r - np.dot(P[i, :], Q[j, :].T)
    # Regularization을 반영한 SGD 업데이트 공식 적용
    P[i, :] = P[i, :] + learning_rate * (eji * Q[j, :] - r_lambda * P[i, :])
    Q[j, :] = Q[j, :] + learning_rate * (eji * P[i, :] - r_lambda * Q[j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0:
      print('### iteration step: ', step, 'rmse: ', rmse)

### iteration step:  0 rmse:  3.261355059488935
### iteration step:  0 rmse:  3.26040057174686
### iteration step:  0 rmse:  3.253984404542389
### iteration step:  0 rmse:  3.2521583839863624
### iteration step:  0 rmse:  3.252335303789125
### iteration step:  0 rmse:  3.251072196430487
### iteration step:  0 rmse:  3.2492449982564864
### iteration step:  0 rmse:  3.247416477570409
### iteration step:  0 rmse:  3.241926055455223
### iteration step:  0 rmse:  3.2400454107613084
### iteration step:  0 rmse:  3.240166740749792
### iteration step:  0 rmse:  3.2388050277987723
### iteration step:  50 rmse:  0.5003190892212748
### iteration step:  50 rmse:  0.5001616291326989
### iteration step:  50 rmse:  0.49899601202578087
### iteration step:  50 rmse:  0.4988483450145831
### iteration step:  50 rmse:  0.49895189256631756
### iteration step:  50 rmse:  0.49833236830090993
### iteration step:  50 rmse:  0.4984148489378701
### iteration step:  50 rmse:  0.49792599580240876
### iteration ste

- 분해된 P와 Q 함수를 P * Q.T로 예측 행렬을 만들어서 출력

In [4]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


##**9.8 파이썬 추천 시스템 패키지 - Surprise**

###**[Surprise 패키지 소개]**

In [6]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357279 sha256=20c86d0cd1e9477a52ad50850d0e6d2594ea0637e4df88a29ac567a8f1598dbb
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [7]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)


Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


- SVD로 잠재 요인 협업 필터링 수행
- 학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천 수행 - test()와 predict(): Surprise에서 추천을 예측하는 메서드
  - test(): 사용자-아이템 평점 데이터 세트 전체에 대해 추천을 예측하는 메서드, 입력된 데이터 세트에 대해 추천 데이터 세트를 만들어 줌
  - predict(): 개별 사용자와 영화에 대한 추천 평점을 반환해 줌

In [9]:
algo = SVD()
algo.fit(trainset)

predictions = algo.test(testset)
print('prediction type: ', type(predictions), ' size: ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type:  <class 'list'>  size:  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.386932093447826, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8147246721394614, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9161230361701462, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.7186230640100977, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3514339985975967, details={'was_impossible': False})]

- Prediction 객체에서 uid, iid, est 속성을 추출한 예제

In [10]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]


[('120', '282', 3.386932093447826),
 ('882', '291', 3.8147246721394614),
 ('535', '507', 3.9161230361701462)]

- Surprise 패키지의 다른 추천 예측 메서드인 predict()를 이용해 추천 예측


In [11]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.17   {'was_impossible': False}


- Surprise의 accuracy 모듈의 rmse()를 이용해 RMSE 평가 결과를 확인

In [12]:
accuracy.rmse(predictions)

RMSE: 0.9482


0.9481869292906402

###**[Surprise 주요 모듈 소개]**
OS 파일 데이터를 Surprise 데이터 세트로 로딩

In [14]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

ratings = pd.read_csv('drive/My Drive/Colab Notebooks/data/ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 업로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('drive/My Drive/Colab Notebooks/data/ml-latest-small/ratings_noh.csv', index = False, header = False)


Mounted at /content/drive


- ratings_noh.csv 파일은 ratings.csv 파일에서 헤더가 삭제된 파일
- ratings_noh.csv를 Dataset 모듈의 load_from_file()을 이용해 Dataset로 로드

In [16]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale = (0.5, 5))
data = Dataset.load_from_file('drive/My Drive/Colab Notebooks/data/ml-latest-small/ratings_noh.csv', reader = reader)


- SVD 행렬 분해 기법을 이용한 추천 예측
    - 잠재 요인 크기 K 값을 나타내는 파라미터인 n_factors를 50으로 설정해 데이터를 학습한 뒤 테스트 데이터 세트를 적용해 예측 평점 구하기
    - 예측 평점과 실제 평점 데이터를 RMSE로 평가

In [17]:
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors = 50, random_state = 0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

- 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

In [18]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('drive/My Drive/Colab Notebooks/data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 함
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

algo = SVD(n_factors = 50, random_state = 0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

###**[교차 검증과 하이퍼 파라미터 튜닝]**

In [19]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('drive/My Drive/Colab Notebooks/data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state = 0)
cross_validate(algo, data, measures = ['RMSE', 'MSE'], cv = 5, verbose = True)


Evaluating RMSE, MSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8703  0.8748  0.8732  0.8734  0.8720  0.8727  0.0015  
MSE (testset)     0.7574  0.7653  0.7625  0.7628  0.7603  0.7617  0.0026  
Fit time          1.65    2.59    2.00    1.68    1.67    1.92    0.36    
Test time         0.23    0.55    0.13    0.12    0.12    0.23    0.16    


{'test_rmse': array([0.87030739, 0.87482049, 0.87321225, 0.87341209, 0.87196986]),
 'test_mse': array([0.75743495, 0.7653109 , 0.76249963, 0.76284869, 0.76033143]),
 'fit_time': (1.647949457168579,
  2.5882577896118164,
  1.9982025623321533,
  1.6811463832855225,
  1.673377513885498),
 'test_time': (0.23110675811767578,
  0.5467901229858398,
  0.13285279273986816,
  0.12348580360412598,
  0.12383365631103516)}

In [20]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures = ['rmse', 'mae'], cv = 3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8774511791810724
{'n_epochs': 20, 'n_factors': 50}


###**[Surprise를 이용한 개인화 영화 추천 시스템 구축]**

In [22]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류 발생
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']])
algo = SVD(n_factors = 50, random_state = 0)
algo.fit(data)

TypeError: Dataset.load_from_df() missing 1 required positional argument: 'reader'

In [24]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale = (0.5, 5))
# DatasestAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file = 'drive/My Drive/Colab Notebooks/data/ml-latest-small/ratings_noh.csv', reader = reader)

# 전체 데이터를 학습 데이터로 생성함
trainset = data_folds.build_full_trainset()

algo = SVD(n_epochs = 20, n_factors = 50, random_state = 0)
algo.fit(trainset)

# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('drive/My Drive/Colab Notebooks/data/ml-latest-small/movies.csv')

# userId = 9의 movieId 데이터를 추출해 movieId = 42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds == 42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [25]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose = True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [27]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

  # 모든 영화의 movieId를 리스트로 생성
  total_movies = movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies),
        '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [30]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n = 10):

  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise의 Prediction 객체를 원소로 가지고 있음
  # [Prediction(uid ='9', iid = '1', est = 3.69), Prediction(uid = '9', iid = '2', est = 2.98),,,]

  # 이를 est 값으로 정렬하기 위해 아래의 sortkey_est 함수를 정의함
  # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행

  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n 개의 최상위 값 추출
  predictions.sort(key = sortkey_est, reverse = True)
  top_predictions = predictions[:top_n]

  # top_n 으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids,
                                                                       top_movie_titles,
                                                                       top_movie_rating)]


  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n =10)

print('#### TOP-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
  print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
#### TOP-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
